In [1]:
from itertools import permutations
import numpy as np
import time 

In [2]:
print('Enter the name of source file[ip.txt,ip1.txt]: ')
file_path = input()
with open(file_path,'rt') as fin:
    lines = fin.readlines()
    n = int(lines[0])
    e = int(lines[1])


    adjacency = np.array([[False for j in range(n)] for i in range(n)])

    for i in range(e):
        data = tuple(map(int,lines[i+2].split(' ')))
        adjacency[data[0]-1][data[1]-1] = adjacency[data[1]-1][data[0]-1] = True

Enter the name of source file[ip.txt,ip1.txt]: 
ip1.txt


## Contents of file



In [3]:
!cat $file_path

12
20
1 2
1 3
1 4
2 5
2 6
5 6
5 9
5 8
6 9
6 12
12 11
9 11
11 8
11 10
8 10
10 7
7 8
7 3
8 4
3 4



In [4]:
adjacency.shape[0]

12

In [5]:
adjacency

array([[False,  True,  True,  True, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False,  True,  True, False, False, False,
        False, False, False],
       [ True, False, False,  True, False, False,  True, False, False,
        False, False, False],
       [ True, False,  True, False, False, False, False,  True, False,
        False, False, False],
       [False,  True, False, False, False,  True, False,  True,  True,
        False, False, False],
       [False,  True, False, False,  True, False, False, False,  True,
        False, False,  True],
       [False, False,  True, False, False, False, False,  True, False,
         True, False, False],
       [False, False, False,  True,  True, False,  True, False, False,
         True,  True, False],
       [False, False, False, False,  True,  True, False, False, False,
        False,  True, False],
       [False, False, False, False, False, False,  True,  True, False,
        False,  True

In [6]:
def test(sequence,adjacency):
    length = len(sequence)
    if length != n + 1:
        return False

    for i in range(length - 1):
        if not adjacency[sequence[i]][sequence[i+1]]:
            return False

    return True

In [7]:
def hamilton_permutations(adjacency):
    global n
    found = False
    for seq in permutations(range(1,adjacency.shape[0])):
        seq = tuple([0]) + seq + tuple([0])
        if(test(seq,adjacency)):
            found = True
            actual_seq = tuple(map(lambda x: x + 1,seq))
            print('Circuit Exists: ',actual_seq)
    if not found:
        print('No Hamiltonian Circuit found!')
     
    
    

In [8]:
start = time.time()
hamilton_permutations(adjacency)
end = time.time()
print('Time taken: ',(end-start))

Circuit Exists:  (1, 2, 5, 9, 6, 12, 11, 8, 10, 7, 3, 4, 1)
Circuit Exists:  (1, 2, 5, 9, 6, 12, 11, 10, 7, 8, 4, 3, 1)
Circuit Exists:  (1, 2, 5, 9, 6, 12, 11, 10, 8, 7, 3, 4, 1)
Circuit Exists:  (1, 2, 6, 12, 11, 9, 5, 8, 10, 7, 3, 4, 1)
Circuit Exists:  (1, 3, 4, 8, 7, 10, 11, 12, 6, 9, 5, 2, 1)
Circuit Exists:  (1, 4, 3, 7, 8, 10, 11, 12, 6, 9, 5, 2, 1)
Circuit Exists:  (1, 4, 3, 7, 10, 8, 5, 9, 11, 12, 6, 2, 1)
Circuit Exists:  (1, 4, 3, 7, 10, 8, 11, 12, 6, 9, 5, 2, 1)
Time taken:  36.663930892944336


In [9]:
def hamilton_pruning(adjacency):
    visited = np.array([False for i in range(adjacency.shape[0])])
    sequence = tuple([0])
    
    def generate_successors(current_state):
        
        #All vertices are present exactly once, so add start vertex to path
        if len(current_state) == adjacency.shape[0]:
            
            #Check if the start vertex is adjacent to the last vertex in path
            if adjacency[current_state[-1]][0]:
                yield current_state + tuple([0])
            #If not, Not a valid path
            else:
                yield None

            return
        
        
        #Finding the last vertex in the path
        last_vertex  = current_state[-1]
        
        #Acquiring the adjacent vertices
        adjacent_vertices = adjacency[last_vertex]
        
        found = False
        
        for vertex in range(len(adjacent_vertices)):
            
            #Add vertices adjacent to this vertex which are not previously in the path
            if adjacent_vertices[vertex] and vertex not in current_state:
                yield current_state + tuple([vertex])
                found = True
                
        #The last vertex in sequence contains no adjacent vertices which are not in path
        if not found:
            yield None
        
    
    def hamilton_dfs(sequence):

        #If it is hamiltonian path,print and exit
        if(test(sequence,adjacency)):
            actual_seq = tuple(map(lambda x: x + 1,sequence))
            print('Circuit Exists: ',actual_seq)
            return
        
        #Add another vertex to the state
        for next_state in generate_successors(sequence):
            
            #
            if next_state is None:
                break
            hamilton_dfs(next_state)
        

    hamilton_dfs(sequence)
    

In [10]:
start = time.time()
hamilton_pruning(adjacency)
end = time.time()
print('Time taken: ',(end-start))

Circuit Exists:  (1, 2, 5, 9, 6, 12, 11, 8, 10, 7, 3, 4, 1)
Circuit Exists:  (1, 2, 5, 9, 6, 12, 11, 10, 7, 8, 4, 3, 1)
Circuit Exists:  (1, 2, 5, 9, 6, 12, 11, 10, 8, 7, 3, 4, 1)
Circuit Exists:  (1, 2, 6, 12, 11, 9, 5, 8, 10, 7, 3, 4, 1)
Circuit Exists:  (1, 3, 4, 8, 7, 10, 11, 12, 6, 9, 5, 2, 1)
Circuit Exists:  (1, 4, 3, 7, 8, 10, 11, 12, 6, 9, 5, 2, 1)
Circuit Exists:  (1, 4, 3, 7, 10, 8, 5, 9, 11, 12, 6, 2, 1)
Circuit Exists:  (1, 4, 3, 7, 10, 8, 11, 12, 6, 9, 5, 2, 1)
Time taken:  0.012526273727416992
